# Experiment 04

This Experiment is the main experiment of the work, consists in run an optimized Random Forest in order to try to predict the values in question.(https://www.researchgate.net/publication/221533996_From_an_artificial_neural_network_to_a_stock_market_day-trading_system_A_case_study_on_the_BMF_BOVESPA).

In [1]:
import numpy as np
import pandas as pd

import Utils 
from Utils import calculateEMA, calculateBB, addPreviousDays, removeColumns, getPeriod, mape, mae, createDate, getDate, printResult

import matplotlib.pyplot as plt
import matplotlib.dates as mpdates
import mplfinance as mpf
from mplfinance.original_flavor import candlestick_ohlc

import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestRegressor

### Constants and Parameters

In [2]:
dfColumns = ['date','open', 'high', 'low','close','tickvol','volume','spread']
newColumns = ['open', 'high', 'low', 'close']
col15 = ['open','d-5_high','d-5_low','d-4_high','d-4_low','d-3_high','d-3_low','d-2_high','d-2_low','d-1_high','d-1_low','EMA_high','EMA_low','UB_close','LB_close']
col25 = ['open','d-5_open','d-5_high','d-5_low','d-5_close','d-4_open','d-4_high','d-4_low','d-4_close','d-3_open','d-3_high','d-3_low','d-3_close','d-2_open','d-2_high','d-2_low','d-2_close','d-1_open','d-1_high','d-1_low','d-1_close','EMA_high' ,'EMA_low','UB_close','LB_close']
DAYS = 5

In [3]:
TEST_BEGIN = '2020.06.01'
TEST_END = '2021.06.01'

WINDOW = 200
DRIFT = 10

TIMES = 1

def safeDate(df, begin, period):
    index = df[df['date'] == begin].index[0]
    
    newIndex = index - period*3
    newDate = df[df.index == newIndex].date
    
    return newDate.iloc[0]

### Opening the dataset

In [4]:
df = pd.read_csv('../Data/PETR4_Daily.csv', sep = "\t")
df.columns = dfColumns
df = removeColumns(df, ['tickvol', 'volume','spread'])

beginDate = safeDate(df, TEST_BEGIN, WINDOW)

df2 = pd.read_csv('../Data/VALE3_Daily.csv', sep = "\t")
df2.columns = dfColumns
df2 = removeColumns(df2, ['tickvol', 'volume','spread'])

In [5]:
#PETR4
df = calculateEMA(df,DAYS,newColumns)
df = calculateBB(df,DAYS,newColumns)
df = addPreviousDays(df,DAYS, newColumns)

#VALE3
df2 = calculateEMA(df2,DAYS,newColumns)
df2 = calculateBB(df2,DAYS,newColumns)
df2 = addPreviousDays(df2,DAYS, newColumns)

In [6]:
initialDf_PETR4 = getPeriod(df, beginDate ,TEST_END, True)
initialDf_VALE3 = getPeriod(df2,beginDate,TEST_END, True)

In [7]:
initialDf_PETR4

,date,open,high,low,close,EMA_open,EMA_high,EMA_low,EMA_close,UB_open,...,d-3_low,d-3_close,d-4_open,d-4_high,d-4_low,d-4_close,d-5_open,d-5_high,d-5_low,d-5_close
0,2017.12.21,13.89,14.39,13.88,14.38,13.828810,14.066436,13.763798,13.973471,13.945436,...,13.72,13.80,13.65,13.82,13.56,13.56,13.70,13.89,13.60,13.61
1,2017.12.22,14.28,14.41,14.23,14.28,13.979207,14.180957,13.919198,14.075647,14.338318,...,13.66,13.73,13.75,13.90,13.72,13.80,13.65,13.82,13.56,13.56
2,2017.12.26,14.28,14.50,14.23,14.48,14.079471,14.287305,14.022799,14.210431,14.515586,...,13.76,13.82,13.77,13.83,13.66,13.73,13.75,13.90,13.72,13.80
3,2017.12.27,14.50,14.64,14.49,14.56,14.219648,14.404870,14.178533,14.326954,14.738558,...,13.88,14.38,13.80,13.88,13.76,13.82,13.77,13.83,13.66,13.73
4,2017.12.28,14.60,14.63,14.51,14.60,14.346432,14.479913,14.289022,14.417970,14.856260,...,14.23,14.28,13.89,14.39,13.88,14.38,13.80,13.88,13.76,13.82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
844,2021.05.26,25.91,26.18,25.65,26.09,26.114791,26.354219,25.760657,26.042263,26.608719,...,25.94,25.95,26.15,26.28,25.72,25.93,25.85,26.35,25.80,26.15
845,2021.05.27,26.08,26.25,25.77,25.91,26.103194,26.319479,25.763771,25.998175,26.604087,...,25.99,26.39,26.08,26.31,25.94,25.95,26.15,26.28,25.72,25.93
846,2021.05.28,26.20,27.02,26.15,26.99,26.135462,26.552986,25.892514,26.328784,26.617551,...,25.78,25.84,26.28,26.52,25.99,26.39,26.08,26.31,25.94,25.95
847,2021.05.31,26.96,27.20,26.78,26.87,26.410308,26.768657,26.188343,26.509189,27.147657,...,25.65,26.09,26.48,26.62,25.78,25.84,26.28,26.52,25.99,26.39


In [8]:
initialDf_VALE3

,date,open,high,low,close,EMA_open,EMA_high,EMA_low,EMA_close,UB_open,...,d-3_low,d-3_close,d-4_open,d-4_high,d-4_low,d-4_close,d-5_open,d-5_high,d-5_low,d-5_close
0,2017.12.21,34.21,34.76,34.19,34.76,33.157472,33.771736,33.085643,33.654289,34.663427,...,32.67,33.11,31.95,32.62,31.81,32.28,31.42,32.01,31.32,31.71
1,2017.12.22,34.73,35.17,34.63,35.13,33.681648,34.237824,33.600429,34.146193,35.263251,...,32.93,33.12,32.80,33.27,32.67,33.11,31.95,32.62,31.81,32.28
2,2017.12.26,34.97,35.32,34.45,35.18,34.111099,34.598549,33.883619,34.490795,35.793179,...,33.22,34.26,33.00,33.34,32.93,33.12,32.80,33.27,32.67,33.11
3,2017.12.27,35.14,35.50,34.99,35.19,34.454066,34.899033,34.252413,34.723863,36.001682,...,34.19,34.76,33.22,34.31,33.22,34.26,33.00,33.34,32.93,33.12
4,2017.12.28,35.33,35.55,35.25,35.55,34.746044,35.116022,34.584942,34.999242,35.741748,...,34.63,35.13,34.21,34.76,34.19,34.76,33.22,34.31,33.22,34.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
844,2021.05.26,106.37,110.32,106.03,110.20,109.166674,111.131617,107.622606,109.807900,113.973861,...,108.50,109.40,111.70,112.10,110.20,111.11,112.10,112.78,109.84,112.25
845,2021.05.27,110.80,112.09,109.77,110.99,109.711116,111.451078,108.338404,110.201933,113.405981,...,107.51,109.78,111.11,111.25,108.50,109.40,111.70,112.10,110.20,111.11
846,2021.05.28,111.11,112.75,109.42,111.59,110.177411,111.884052,108.698936,110.664622,113.405981,...,106.78,107.05,108.00,110.14,107.51,109.78,111.11,111.25,108.50,109.40
847,2021.05.31,112.50,114.89,112.50,114.78,110.951607,112.886035,109.965957,112.036415,114.811204,...,106.03,110.20,110.28,110.89,106.78,107.05,108.00,110.14,107.51,109.78


#### Experimento 04: Utilizando Random Forest

In [12]:
def getTrainPeriod(df, begin, period):
    
    index = df[df['date'] == begin].index[0]
    index = index-1
    
    endDate = df[df.index == index].date
    beginDate = df[df.index == (index-period)].date
    
    return beginDate.iloc[0], endDate.iloc[0]
    

def exp04(df, stock, inputColumns = []):
    
    beginTrain, endTrain = getTrainPeriod(df,TEST_BEGIN, WINDOW)
    
    df_test_aux = getPeriod(df, TEST_BEGIN, TEST_END)
    
    indexBegin = df_test_aux.head(1).index[0]
    indexEnd = df_test_aux.tail(1).index[0]
    
    for i in range (indexBegin, indexEnd, DRIFT):
        print(i)
        
        df_test = df[(df.index >= i) & (df.index <= i+DRIFT-1)]
        df_train = df[(df.index >= i-WINDOW) & (df.index < i)]
    
        if(inputColumns == []):
            x_train = df_train.drop(['date', 'high', 'low', 'close'], axis=1)
            y_train = df_train[['high','low']]

            x_test = df_test.drop(['date', 'high', 'low', 'close'], axis=1)
            y_test = df_test[['high','low']]
        else:
            x_train = df_train[inputColumns]
            y_train = df_train[['high','low']]

            x_test = df_test[inputColumns]
            y_test = df_test[['high','low']]

        rf = RandomForestRegressor(n_estimators = 500)
        rf.fit(x_train, y_train);
        y_pred = rf.predict(x_test)
        
        if(i == indexBegin):
            predDf = y_test.copy()
            predDf['high_pred'] = y_pred[:,0]
            predDf['low_pred'] = y_pred[:,1]
        else:
            auxDf = y_test.copy()
            auxDf['high_pred'] = y_pred[:,0]
            auxDf['low_pred'] = y_pred[:,1]
            predDf = predDf.append(auxDf) 
    
        #print (mape(y_test['high'],y_pred[:,0]))
        #print (mape(y_test['low'],y_pred[:,1]))
        #print (mae(y_test['high'],y_pred[:,0]))
        #print (mae(y_test['low'],y_pred[:,1]))
    
    
    errorMax = mape(predDf['high'],predDf['high_pred'])
    errorMin = mape(predDf['low'],predDf['low_pred'])
    
    return errorMax, errorMin, predDf
    
errorMax_petr4, errorMin_petr4, predDf_petr4 = exp04(initialDf_PETR4, 'PETR4')
errorMax_vale3, errorMin_vale3, predDf_vale3 = exp04(initialDf_VALE3, 'VALE3')

predDf_petr4['date'] = getPeriod(initialDf_PETR4, TEST_BEGIN, TEST_END)['date']
predDf_vale3['date'] = getPeriod(initialDf_PETR4, TEST_BEGIN, TEST_END)['date']

600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840


In [10]:
print('Erro Petr4:')
printResult(errorMax_petr4, 'high')
printResult(errorMin_petr4, 'low')
print('\nErro vale3:')
printResult(errorMax_vale3, 'high')
printResult(errorMin_vale3, 'low')

Erro Petr4:
high: 1.74 +- 1.82 %
low: 1.87 +- 2.32 %

Erro vale3:
high: 2.41 +- 2.79 %
low: 2.45 +- 2.7 %


In [11]:
display(predDf_petr4.head(10))
predDf_petr4.to_pickle('../Data/PETR4_Pred_ytd.pkl')
display(predDf_vale3.head(10))
predDf_vale3.to_pickle('../Data/VALE3_Pred_ytd.pkl')

,high,low,high_pred,low_pred,date
600,19.89,19.35,19.46755,18.66315,2020.06.01
601,20.70,19.93,19.46890,18.66245,2020.06.02
602,21.20,20.71,19.83215,19.00570,2020.06.03
603,21.07,20.36,19.74140,18.89875,2020.06.04
604,22.28,21.34,19.93865,19.08815,2020.06.05
605,21.86,21.29,20.73650,19.83735,2020.06.08
606,21.32,20.94,21.56790,20.61715,2020.06.09
607,21.19,20.32,21.33520,20.38565,2020.06.10
608,20.48,19.45,19.90345,19.05030,2020.06.12
609,20.40,18.91,19.58675,18.76710,2020.06.15


,high,low,high_pred,low_pred,date
600,50.40,49.14,49.73290,48.67650,2020.06.01
601,50.35,49.60,50.51375,49.58665,2020.06.02
602,50.86,49.96,50.74020,49.89305,2020.06.03
603,52.13,50.09,51.01740,50.13460,2020.06.04
604,53.07,50.86,52.26270,51.18830,2020.06.05
605,52.08,50.52,52.10685,50.99675,2020.06.08
606,51.47,50.65,52.12025,50.84390,2020.06.09
607,51.66,50.31,52.24530,50.86215,2020.06.10
608,50.59,48.65,52.03815,50.71490,2020.06.12
609,50.94,48.79,50.89725,49.55625,2020.06.15
